# 网络搜索

> 没有一条帆船能吹动自己行驶，它需要外面的风。 -- 因可觅[《量子离歌》](https://read.douban.com/ebook/12945743/)

网络搜索本身是一个很重的服务，它包含三项核心任务：网页爬取、网页索引和网页检索。**网页索引** 几乎只能由大公司完成。即使 [SearXNG](https://github.com/searxng/searxng) 这样的开源检索方案，也只是聚合第三方搜索引擎（如必应、搜狗）的结果，没有独立索引能力。因此，即使独立开发网络搜索功能，也难以绕开大公司的服务。不如一开始就老老实实用大公司的 API，来获取搜索结果。

本节介绍三种网络搜索方式：

- 使用 [DashScope](https://dashscope.aliyun.com/)
- 使用 [Tavily](https://www.tavily.com/)
- 使用 [DDGS](https://github.com/deedy5/ddgs)

**DashScope** 是阿里的 MaaS 平台，由于我们已经有了阿里的 `API_KEY`，这对我们是最轻松的方式。**Tavily** 是 LangChain [推荐](https://docs.langchain.com/oss/python/integrations/retrievers/tavily) 的搜索服务，有免费额度，但需要花时间申请 `API_KEY`。**DDGS** 是一个免费的搜索引擎，不需要 `API_KEY`，但搜索结果的质量较差。

下面来看这三种搜索服务的使用方法和效果。

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool

# 加载模型配置
_ = load_dotenv()

# 加载模型
llm = ChatOpenAI(
    model="qwen-max",
    temperature=0.7,
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
)

In [2]:
# !pip install dashscope langchain-community tavily-python ddgs

## 一、使用 DashScope

首先安装 dashscope：

```bash
pip install dashscope
```

只需要将 `Generation.call` 函数的网络参数 `enable_search` 设为 `True` 就能接入互联网。下面把这个函数做成工具供 Agent 调用。

In [3]:
from dashscope import Generation

@tool
def dashscope_search(query: str) -> str:
    """
    使用夸克搜索 API 搜索互联网信息。
    """
    response = Generation.call(
        model='qwen3-max',
        prompt=query,
        enable_search=True,
        result_format='message'
    )

    if response.status_code == 200:
        return response.output.choices[0].message.content
    else:
        return (
            "Search failed with status code: "
            f"{response.status_code}, message: {response.message}"
        )

# 创建 Agent
agent = create_agent(
    model=llm,
    tools=[dashscope_search],
    system_prompt="你是一个智能助手，回答前必须使用工具搜索互联网信息",
)

# 运行 Agent
response = agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "告诉我今天的日期，以及今天最重要的一条新闻"
    }]}
)

In [4]:
# 获取 Agent 的全部回复
for message in response['messages']:
    message.pretty_print()

================================ Human Message =================================

告诉我今天的日期，以及今天最重要的一条新闻
================================== Ai Message ==================================
Tool Calls:
  dashscope_search (call_0d753c0e020d457f9309a5)
 Call ID: call_0d753c0e020d457f9309a5
  Args:
    query: 今天的日期
  dashscope_search (call_d714a572adba4037b921eb)
 Call ID: call_d714a572adba4037b921eb
  Args:
    query: 今天最重要的一条新闻
================================= Tool Message =================================
Name: dashscope_search

今天的日期是：**2025年12月28日，星期日**。
================================= Tool Message =================================
Name: dashscope_search

截至**2025年12月28日**，今天最重要的一条新闻是：

### **美国宣布将对中国部分半导体产品加征301关税，但设置18个月缓冲期（至2027年6月）**

#### 核心要点：
- **事件背景**：美东时间12月23日，美国贸易代表办公室发布针对中国半导体政策的“301调查”结果，认定中方相关产业政策“损害美国利益”，决定对包括晶体管、集成电路、二极管、放大器等关键半导体产品加征关税。
- **特殊安排**：**当前税率为0%**，但**2027年6月起将提高税率**。这一“延迟生效”机制被广泛解读为美方在施压与维稳之间寻求平衡。
- **中方回应**：12月25日，中国商务部发言人**何咏前**在例行发布会上**严正交涉**，强调：
  - 不认

## 二、使用 Tavily

首先安装 tavily：

```bash
pip install langchain-community tavily-python
```

然后从 [Tavily](https://www.tavily.com/) 官网申请 `API_KEY`，填入下面的方框中。

In [5]:
import getpass
os.environ["TAVILY_API_KEY"] = getpass.getpass()

 ········


In [12]:
from langchain_community.retrievers import TavilySearchAPIRetriever

retriever = TavilySearchAPIRetriever(k=3)
query = "告诉我今天的日期，以及今天最重要的一条新闻"

返回的内容有点多，可以解开下面这行命令的注释查看👇

In [13]:
# retriever.invoke(query)

同样把 Tavily 作为工具加入 Agent 看看效果怎么样！

In [14]:
prompt_template = """基于下面的搜索结果回答问题：

搜索结果：{context}

问题：{question}"""

def format_docs(docs):
    """合并 Tavily 的搜索结果"""
    return "\n\n".join(doc.page_content for doc in docs)

@tool
def tavily_search(query: str) -> str:
    """
    使用 Tavily 搜索 API 搜索互联网信息。
    """
    retriever = TavilySearchAPIRetriever(k=3)
    contents = retriever.invoke(query)
    if len(contents) > 0:
        return format_docs(contents)
    else:
        return "没有搜索结果"

# 创建 Agent
agent = create_agent(
    model=llm,
    tools=[tavily_search],
    system_prompt="你是一个智能助手，回答前必须使用工具搜索互联网信息",
)

# 运行 Agent
response = agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "告诉我今天的日期，以及今天最重要的一条新闻"
    }]}
)

In [15]:
# 获取 Agent 的全部回复
for message in response['messages']:
    message.pretty_print()

================================ Human Message =================================

告诉我今天的日期，以及今天最重要的一条新闻
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_0365b8f5023d445ea009dd)
 Call ID: call_0365b8f5023d445ea009dd
  Args:
    query: 今天的日期
================================= Tool Message =================================
Name: tavily_search

# 今天的日期. ## 今日及下一个节日. ### 元旦. ### 春节. ### 劳动节. ### 端午节. ## 当前日期概览. ### MM-DD-YYYY (US). ### DD-MM-YYYY (EU). ### 周起始日. ### 周末. ## 额外日期指标. ### 日期编号. ### ISO周日期. ### 月份. ### 每月天数. ### 年份. ## 跨日历的今天日期. ### 佛教. ### 中文. ### 科普特. ### 埃塞俄比亚. ### 埃塞俄比克. ### 希伯来. ### 印度. ### 伊斯兰教. ### 日本. ### 波斯. ### 中华民国. ## 国际日期表示标准. **日期格式变体**。国际上普遍存在各种表示当前日期的方法，其中最为人熟知的是小端（dd-mm-yyyy）、中端（mm-dd-yyyy）和大端（yyyy-mm-dd）格式。. **美国日期格式的古怪之处**。美国独特地使用mm-dd-yyyy日期格式，而大多数国家采用dd-mm-yyyy约定。这种分歧归因于与英国的历史联系，在那里月份优先格式在与欧洲标准对齐之前盛行。美国人保留了这种偏好，自那时以来一直保持这种格式。. **日期呈现标准**。日期表示中固有的歧义，例如将05-06-2024解释为6月5日或5月6日，给读者带来挑战。为了减轻这种歧义，国际标

## 三、使用 DDGS

首先安装 DDGS：

```bash
pip install ddgs
```

我们直接使用 DDGS 获取的摘要信息，没有做进一步网页爬取。下面将 DDGS 做成工具。

In [16]:
from ddgs import DDGS

# 创建 ddgs 客户端
ddgs = DDGS()

@tool
def ddgs_search(query: str) -> str:
    """
    使用 DDGS 搜索 API 搜索互联网信息。

    Args:
        query: 搜索关键词或问题。
        max_results: 返回的最大结果条数。

    Returns:
        包含每条搜索结果的标题、摘要与链接的字符串。
    """
    results = list(
        ddgs.text(
            query=query,
            region="wt-wt",  # wt-wt zh
            timelimit='y',
            safesearch='off',  # moderate off
            page=1,
            backend='auto',
            max_results=3,
        )
    )

    content = ""
    for i, r in enumerate(results, 1):
        content += f"【结果 {i}】\n"
        content += f"标题: {r['title']}\n"
        content += f"摘要: {r['body']}\n"
        content += f"链接: {r['href']}\n\n"

    return content

# 创建 Agent
agent = create_agent(
    model=llm,
    tools=[ddgs_search],
    system_prompt="你是一个智能助手，回答前必须使用工具搜索互联网信息",
)

# 运行 Agent
response = agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "告诉我今天的日期，以及今天最重要的一条新闻"
    }]}
)

In [17]:
# 获取 Agent 的全部回复
for message in response['messages']:
    message.pretty_print()

================================ Human Message =================================

告诉我今天的日期，以及今天最重要的一条新闻
================================== Ai Message ==================================
Tool Calls:
  ddgs_search (call_50f65b36c4b04cac8a7278)
 Call ID: call_50f65b36c4b04cac8a7278
  Args:
    query: 今天的日期
    max_results: 1
  ddgs_search (call_3519d758f10f4254ac7b9e)
 Call ID: call_3519d758f10f4254ac7b9e
  Args:
    query: 今天 最重要的新闻
    max_results: 1
================================= Tool Message =================================
Name: ddgs_search

【结果 1】
标题: excel怎么自动计算某日期到今天的时间间隔-百度经验
摘要: May 18, 2019 · 首先要了解TODAY（）函数。在excel或wps中，TODAY（）可以返回今天的日期值。这里举例，可以看到A1中自动返回了2019/5/18
链接: https://jingyan.baidu.com/article/d45ad14869204069552b802d.html

【结果 2】
标题: 今天 - 維基百科，自由的百科全書
摘要: 
链接: 

【结果 3】
标题: Excel表格如何插入日期和时间并实现自动更新 - 百度经验
摘要: Jan 20, 2021 · 在数据录入过程中，有时需要录入当前日期、时间，并实现自动更新，使用NOW函数和TODAY函数可以轻松实现。 NOW函数可以实现返回当前日期和时间所对应的序列号。 …
链接: https://jingyan.baidu.com/article/bad08e1e86348348c85121f2.h